In [ ]:
from kafka import KafkaProducer
import yfinance as yf
import json
import time

In [ ]:
# Define the Kafka producer
producer = KafkaProducer(
    bootstrap_servers='10.204.131.11:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

TICKERS = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'V']

In [ ]:
# fetch real-time data for all tickers
def fetch_data():
    data = {}
    for ticker in TICKERS:
        ticker_data = yf.download(ticker, interval="1m", period="1d")
        # Convert the last row of the DataFrame to a dictionary
        last_row = ticker_data.tail(1).to_dict(orient='records')[0]
        data[ticker] = last_row
    return data

In [ ]:
try:
    while True:
        # Fetch data for all tickers
        data = fetch_data()

        # Prepare and send data to Kafka
        for ticker, ticker_data in data.items():
            # Convert data to JSON
            data_json = json.dumps({ticker: ticker_data})
            # Send data to Kafka topic
            producer.send('g05in', data_json)
        print(data)
        # Wait for a minute before fetching new data
        time.sleep(60)
except KeyboardInterrupt:
    # Flush and close the producer
    producer.flush()
    producer.close()

{'AAPL': {'Open': 192.16000366210938, 'High': 192.47999572753906, 'Low': 192.05999755859375, 'Close': 192.47000122070312, 'Adj Close': 192.47000122070312, 'Volume': 2016314}, 'MSFT': {'Open': 413.94000244140625, 'High': 415.7300109863281, 'Low': 413.94000244140625, 'Close': 415.3074951171875, 'Adj Close': 415.3074951171875, 'Volume': 1296300}, 'GOOG': {'Open': 173.85000610351562, 'High': 174.07000732421875, 'Low': 173.67999267578125, 'Close': 174.0399932861328, 'Adj Close': 174.0399932861328, 'Volume': 792362}, 'AMZN': {'Open': 176.19500732421875, 'High': 176.57000732421875, 'Low': 176.01499938964844, 'Close': 176.46499633789062, 'Adj Close': 176.46499633789062, 'Volume': 1573871}, 'V': {'Open': 272.3599853515625, 'High': 272.760009765625, 'Low': 271.9800109863281, 'Close': 272.6099853515625, 'Adj Close': 272.6099853515625, 'Volume': 362174}}